In [4]:
import requests
from bs4 import BeautifulSoup
import re
import string
from time import sleep
import random
import pickle
import urllib
import urlopen
import os
from hashlib import sha1  

200


In [16]:
def get_songs(proxies, user_agents, artists):
    base = 'http://www.azlyrics.com/b/badbunny.html'
    art_song_dict = {}
    for artist in artists:    
        url = base + artist[0] + '/' + artist + '.html'
        sleep(random.randint(0,10))
        response = requests.get(url, headers = {'User-Agent': random.choice(user_agents)}, proxies=random.choice(proxies))
        soup = BeautifulSoup(response.content, 'lxml')
        lang = []
        for song in soup.findAll(target='_blank'):
            lang.append(str(song.text))
        art_song_dict[artist] = lang
        pickle.dump(art_song_dict, open(artist + '_songs.pickle', 'wb'))
    return art_song_dict

In [20]:
def get_url_AZ(artist, song):
       
    song = re.sub('['+string.punctuation+']', '', song).replace(' ','').lower()
    base = 'http://www.azlyrics.com/lyrics/'
    rest = artist + '/' + song + '.html'
    return base + rest

In [22]:
print(get_url_AZ)

<function get_url_AZ at 0x063EDB28>


In [23]:
def get_lyric_AZ(url, proxies, user_agents):

    if load_local(url):
        soup = BeautifulSoup(load_local(url), 'lxml')
        page_lyric = soup.findAll(style="margin-left:10px;margin-right:10px;")
        lyric = re.sub('[(<.!,;?>/\-)]', " ",  str(page_lyric)).split()
        lyric = [word for word in lyric if word != 'br']
        return lyric[10:-4]
    else:
        sleep(random.randint(0,20))
        response = requests.get(url, headers = {'User-Agent': random.choice(user_agents)}, proxies = random.choice(proxies))
        store_local(url, response.content)
        soup = BeautifulSoup(response.content, 'lxml')
        page_lyric = soup.findAll(style="margin-left:10px;margin-right:10px;")
        lyric = re.sub('[(<.!,;?>/\-)]', " ",  str(page_lyric)).split()
        lyric = [word for word in lyric if word != 'br']
        return lyric[10:-4]

In [25]:
CACHE_DIR = os.path.join(os.path.dirname("Documents\GitHub\data_studio\code\Text Mining Bad Bunny"), 'cache')

In [26]:
def url_to_filename(url):
    hash_file = sha1(url).hexdigest() + '.html'
    return os.path.join(CACHE_DIR, hash_file)

In [27]:
def store_local(url, content):
    if not os.path.isdir(CACHE_DIR):
        os.makedirs(CACHE_DIR)
    local_path = url_to_filename(url)
    with open(local_path, 'wb') as f:
        f.write(content)

In [28]:
def load_local(url):
    local_path = url_to_filename(url)
    if not os.path.exists(local_path):
        return None

    with open(local_path, 'rb') as f:
        return f.read()

In [31]:
def main():
    master_dict = {}
    avg_dict = {}
    with open('rush_songs.pickle', 'rb') as f:
        art_info = pickle.load(f) 
    for artist, songs in art_info.iteritems():
        avg = []
        lyrics = []
        for song in songs:
            print (artist, song)
            url = get_url_AZ(artist,song)
            lyric = get_lyric_AZ(url, proxies, user_agents)
            print (lyric)
            avg.append(len(lyric))
            lyrics.append(lyric)
        avg_dict[artist] = avg
        master_dict[artist] = lyrics
        print (artist, "-------------------------completed----------------------------------")
        pickle.dump(avg_dict, open(artist + '_count.pickle', 'wb'))
        pickle.dump(master_dict, open(artist + '_lyrics.pickle', 'wb'))
    return 'Fully Completed!'

In [32]:
main()

FileNotFoundError: [Errno 2] No such file or directory: 'rush_songs.pickle'